# Projet2:Classification,Accorder ou pas un crédit  à un client bancaire

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
projet = pd.read_csv('creditBanc.csv')

In [4]:
projet.head(100)

,Client_ID,Sexe,Situation_familiale,Personnes_charge,Diplome,Travailleur_independant,Revenu_imposable,Revenu_forfitaire,Montant_pret,Duree_pret,Hitorique_credit,Residence,Decision
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LP001326,Male,No,0.0,Graduate,NaN,6782,0.0,NaN,360.0,NaN,Urban,N
96,LP001327,Female,Yes,0.0,Graduate,No,2484,2302.0,137.0,360.0,1.0,Semiurban,Y
97,LP001333,Male,Yes,0.0,Graduate,No,1977,997.0,50.0,360.0,1.0,Semiurban,Y
98,LP001334,Male,Yes,0.0,Not Graduate,No,4188,0.0,115.0,180.0,1.0,Semiurban,Y


# Identification des colonnes ayant des valeurs manquantes

In [5]:
projet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Client_ID                614 non-null    object 
 1   Sexe                     601 non-null    object 
 2   Situation_familiale      611 non-null    object 
 3   Personnes_charge         599 non-null    float64
 4   Diplome                  614 non-null    object 
 5   Travailleur_independant  582 non-null    object 
 6   Revenu_imposable         614 non-null    int64  
 7   Revenu_forfitaire        614 non-null    float64
 8   Montant_pret             592 non-null    float64
 9   Duree_pret               600 non-null    float64
 10  Hitorique_credit         564 non-null    float64
 11  Residence                614 non-null    object 
 12  Decision                 614 non-null    object 
dtypes: float64(5), int64(1), object(7)
memory usage: 62.5+ KB


# Obtenir une liste des colonnes avec au moins une donnée manquante

In [6]:
projet.columns[projet.isnull().any()]

Index(['Sexe', 'Situation_familiale', 'Personnes_charge',
       'Travailleur_independant', 'Montant_pret', 'Duree_pret',
       'Hitorique_credit'],
      dtype='object')

# Nombre de données manquantes par collone

In [7]:
projet.isnull().sum()

Client_ID                   0
Sexe                       13
Situation_familiale         3
Personnes_charge           15
Diplome                     0
Travailleur_independant    32
Revenu_imposable            0
Revenu_forfitaire           0
Montant_pret               22
Duree_pret                 14
Hitorique_credit           50
Residence                   0
Decision                    0
dtype: int64

# Supprimer les lignes avec des valeurs manquantes dans les colonnes de variables qualitatives 

In [8]:
projet.dropna(subset = ['Sexe', 'Situation_familiale', 'Travailleur_independant'], inplace = True)

In [9]:
projet

,Client_ID,Sexe,Situation_familiale,Personnes_charge,Diplome,Travailleur_independant,Revenu_imposable,Revenu_forfitaire,Montant_pret,Duree_pret,Hitorique_credit,Residence,Decision
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0.0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3.0,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1.0,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2.0,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


# Remplacer les valeurs manquantes des colonnes numériques par la valeur précedente de la même colonne

In [10]:
projet.fillna(method = 'bfill', inplace = True)

In [11]:
projet

,Client_ID,Sexe,Situation_familiale,Personnes_charge,Diplome,Travailleur_independant,Revenu_imposable,Revenu_forfitaire,Montant_pret,Duree_pret,Hitorique_credit,Residence,Decision
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0.0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3.0,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1.0,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2.0,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [12]:
projet.isnull().sum().any()

False

# Remplacement des mots "Male" et "Female" respectivement par 1 et 0

In [13]:
projet['Sexe'].replace(['Male','Female'], [1,0], inplace = True)

In [14]:
projet

,Client_ID,Sexe,Situation_familiale,Personnes_charge,Diplome,Travailleur_independant,Revenu_imposable,Revenu_forfitaire,Montant_pret,Duree_pret,Hitorique_credit,Residence,Decision
0,LP001002,1,No,0.0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y
1,LP001003,1,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,1,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,1,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,1,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,0,No,0.0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,1,Yes,3.0,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,1,Yes,1.0,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,1,Yes,2.0,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


# Remplacement des mots "Yes" et "No" dans les colonnes "Situation_familiale" et "Travailleur_independant", respectivement par 1 et 0

In [15]:
projet['Situation_familiale'].replace(['Yes','No'], [1,0], inplace = True)
projet['Travailleur_independant'].replace(['Yes','No'], [1,0], inplace = True)

In [16]:
projet

,Client_ID,Sexe,Situation_familiale,Personnes_charge,Diplome,Travailleur_independant,Revenu_imposable,Revenu_forfitaire,Montant_pret,Duree_pret,Hitorique_credit,Residence,Decision
0,LP001002,1,0,0.0,Graduate,0,5849,0.0,128.0,360.0,1.0,Urban,Y
1,LP001003,1,1,1.0,Graduate,0,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,1,1,0.0,Graduate,1,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,1,1,0.0,Not Graduate,0,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,1,0,0.0,Graduate,0,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,0,0,0.0,Graduate,0,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,1,1,3.0,Graduate,0,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,1,1,1.0,Graduate,0,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,1,1,2.0,Graduate,0,7583,0.0,187.0,360.0,1.0,Urban,Y


# Remplacement des mots "Graduate" et "Not Graduate" de la colonne Diplome respectivement par 1 et 0

In [17]:
projet['Diplome'].replace(['Graduate','Not Graduate'], [1,0], inplace = True)

In [18]:
projet

,Client_ID,Sexe,Situation_familiale,Personnes_charge,Diplome,Travailleur_independant,Revenu_imposable,Revenu_forfitaire,Montant_pret,Duree_pret,Hitorique_credit,Residence,Decision
0,LP001002,1,0,0.0,1,0,5849,0.0,128.0,360.0,1.0,Urban,Y
1,LP001003,1,1,1.0,1,0,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,1,1,0.0,1,1,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,1,1,0.0,0,0,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,1,0,0.0,1,0,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,0,0,0.0,1,0,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,1,1,3.0,1,0,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,1,1,1.0,1,0,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,1,1,2.0,1,0,7583,0.0,187.0,360.0,1.0,Urban,Y


# Remplacement des mots "Urban", "Semiurban" et "Rural" de la colonne Residence respectivement par 2,1 et 0

In [19]:
projet['Residence'].replace(['Urban','Semiurban','Rural'], [2,1,0], inplace = True)

In [20]:
projet

,Client_ID,Sexe,Situation_familiale,Personnes_charge,Diplome,Travailleur_independant,Revenu_imposable,Revenu_forfitaire,Montant_pret,Duree_pret,Hitorique_credit,Residence,Decision
0,LP001002,1,0,0.0,1,0,5849,0.0,128.0,360.0,1.0,2,Y
1,LP001003,1,1,1.0,1,0,4583,1508.0,128.0,360.0,1.0,0,N
2,LP001005,1,1,0.0,1,1,3000,0.0,66.0,360.0,1.0,2,Y
3,LP001006,1,1,0.0,0,0,2583,2358.0,120.0,360.0,1.0,2,Y
4,LP001008,1,0,0.0,1,0,6000,0.0,141.0,360.0,1.0,2,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,0,0,0.0,1,0,2900,0.0,71.0,360.0,1.0,0,Y
610,LP002979,1,1,3.0,1,0,4106,0.0,40.0,180.0,1.0,0,Y
611,LP002983,1,1,1.0,1,0,8072,240.0,253.0,360.0,1.0,2,Y
612,LP002984,1,1,2.0,1,0,7583,0.0,187.0,360.0,1.0,2,Y


# Repmlacement de "Y" et "N" de la colonne Decision respectivement par 1 et 0

In [21]:
projet['Decision'].replace(['Y','N'], [1,0], inplace = True)

In [22]:
projet

,Client_ID,Sexe,Situation_familiale,Personnes_charge,Diplome,Travailleur_independant,Revenu_imposable,Revenu_forfitaire,Montant_pret,Duree_pret,Hitorique_credit,Residence,Decision
0,LP001002,1,0,0.0,1,0,5849,0.0,128.0,360.0,1.0,2,1
1,LP001003,1,1,1.0,1,0,4583,1508.0,128.0,360.0,1.0,0,0
2,LP001005,1,1,0.0,1,1,3000,0.0,66.0,360.0,1.0,2,1
3,LP001006,1,1,0.0,0,0,2583,2358.0,120.0,360.0,1.0,2,1
4,LP001008,1,0,0.0,1,0,6000,0.0,141.0,360.0,1.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,0,0,0.0,1,0,2900,0.0,71.0,360.0,1.0,0,1
610,LP002979,1,1,3.0,1,0,4106,0.0,40.0,180.0,1.0,0,1
611,LP002983,1,1,1.0,1,0,8072,240.0,253.0,360.0,1.0,2,1
612,LP002984,1,1,2.0,1,0,7583,0.0,187.0,360.0,1.0,2,1


# Suppression de la colonne "Client_ID"

In [23]:
projet.drop('Client_ID', axis=1, inplace = True)

In [24]:
projet

,Sexe,Situation_familiale,Personnes_charge,Diplome,Travailleur_independant,Revenu_imposable,Revenu_forfitaire,Montant_pret,Duree_pret,Hitorique_credit,Residence,Decision
0,1,0,0.0,1,0,5849,0.0,128.0,360.0,1.0,2,1
1,1,1,1.0,1,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,1,0.0,1,1,3000,0.0,66.0,360.0,1.0,2,1
3,1,1,0.0,0,0,2583,2358.0,120.0,360.0,1.0,2,1
4,1,0,0.0,1,0,6000,0.0,141.0,360.0,1.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0.0,1,0,2900,0.0,71.0,360.0,1.0,0,1
610,1,1,3.0,1,0,4106,0.0,40.0,180.0,1.0,0,1
611,1,1,1.0,1,0,8072,240.0,253.0,360.0,1.0,2,1
612,1,1,2.0,1,0,7583,0.0,187.0,360.0,1.0,2,1


# Selection des colonnes des variables explicatives et de la variable à expliquer (à prédire)

In [25]:
projet.columns

Index(['Sexe', 'Situation_familiale', 'Personnes_charge', 'Diplome',
       'Travailleur_independant', 'Revenu_imposable', 'Revenu_forfitaire',
       'Montant_pret', 'Duree_pret', 'Hitorique_credit', 'Residence',
       'Decision'],
      dtype='object')

In [26]:
X = projet[['Sexe', 'Situation_familiale', 'Personnes_charge', 'Diplome',
       'Travailleur_independant', 'Revenu_imposable', 'Revenu_forfitaire',
       'Montant_pret', 'Duree_pret', 'Hitorique_credit', 'Residence']]

In [27]:
X.head(50)

,Sexe,Situation_familiale,Personnes_charge,Diplome,Travailleur_independant,Revenu_imposable,Revenu_forfitaire,Montant_pret,Duree_pret,Hitorique_credit,Residence
0,1,0,0.0,1,0,5849,0.0,128.0,360.0,1.0,2
1,1,1,1.0,1,0,4583,1508.0,128.0,360.0,1.0,0
2,1,1,0.0,1,1,3000,0.0,66.0,360.0,1.0,2
3,1,1,0.0,0,0,2583,2358.0,120.0,360.0,1.0,2
4,1,0,0.0,1,0,6000,0.0,141.0,360.0,1.0,2
5,1,1,2.0,1,1,5417,4196.0,267.0,360.0,1.0,2
6,1,1,0.0,0,0,2333,1516.0,95.0,360.0,1.0,2
7,1,1,3.0,1,0,3036,2504.0,158.0,360.0,0.0,1
8,1,1,2.0,1,0,4006,1526.0,168.0,360.0,1.0,2
9,1,1,1.0,1,0,12841,10968.0,349.0,360.0,1.0,1


In [28]:
y = projet['Decision']

In [29]:
y

0      1
1      0
2      1
3      1
4      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Decision, Length: 566, dtype: int64

# Décomposition des données en données d'entrainement et données de test 

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 101)
print('Données d\'entrainement:',X_train.shape )
print('Données de test:',X_test.shape)

Données d'entrainement: (424, 11)
Données de test: (142, 11)


# Création du modèle (instanciation) de type KNeighborsClassifier

In [31]:
model = KNeighborsClassifier()

In [32]:
model.fit(X_train, y_train)

KNeighborsClassifier()

# Prédiction

In [33]:
predictions = model.predict(X_test)

# Performance du modèle sur la base des données d'entrainement

In [34]:
model.score(X_train, y_train)

0.7405660377358491

# Performance du modèle sur la base des données de test

In [35]:
model.score(X_test, y_test)

0.6690140845070423

# Optimisation du modèle grace à GridSearchCV

In [36]:
param_grid = {'n_neighbors':np.arange(1,100),
              'weights' : ['uniform', 'distance'],
              'metric' : ['euclidean', 'manhattan', 'minkowski']}

In [37]:
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv = 10)

In [38]:
grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                         'weights': ['uniform', 'distance']})

# Meilleur score

In [39]:
grid.best_score_

0.6957364341085273

# Mailleurs paramètres

In [40]:
grid.best_params_

{'metric': 'euclidean', 'n_neighbors': 27, 'weights': 'uniform'}

# Sauvgarde du modèle

In [41]:
model = grid.best_estimator_

In [42]:
model

KNeighborsClassifier(metric='euclidean', n_neighbors=27)

In [43]:
model.score(X_test, y_test)

0.6549295774647887

In [44]:
model.predict([[1,1,2,1,0,7800,2300,350,480,1,2]])

C:\Users\maham\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


array([1], dtype=int64)